# Speed up computations: optimizing Octave code

> *Dec 20, 2019 — Kai T. Ohlhus &lt;<k.ohlhus@gmail.com>&gt; — [CC BY 4.0](http://creativecommons.org/licenses/by/4.0/)*

> "We should **forget about small efficiencies**, say about 97% of the time: **premature optimization is the root of all evil**.
> Yet we should not pass up our opportunities in that critical 3%"*
>
> &mdash;[Donald Knuth (1974)](https://doi.org/10.1145%2F356635.356640)

## Matrix Operations (Vectorization)

- Matrix addition $C = A + B$.

In [1]:
clear all;
m = 200;
n = 5000;
A = rand (m, n);
B = rand (m, n);

In [2]:
tic;
for i = 1:m
  for j = 1:n
    C1(i,j) = A(i,j) + B(i,j);
  endfor
endfor
t1 = toc

t1 =  10.999


- loops in Octave are slow, **nested loops** even **slower**.
- Octave does not know how big `C1` will be.
  Thus **memory** has to be **allocated and freed** in many steps.

In [3]:
tic;
for i = 1:m
  C2(i,1:n) = A(i,:) + B(i,:);
endfor
t2 = toc
printf("%.2f %%", (-1 + t2 / t1) * 100)

t2 =  0.16322
-98.52 %

In [4]:
tic;
C3 = A + B;
t3 = toc
printf("%.2f %%", (-1 + t3 / t1) * 100)

t3 =  0.0069852
-99.94 %

## Memory Management

- Evaluation of $\;\tan(x) = \frac{sin(x)}{cos(x)}$.
- Octave allows to grow arrays automatically `y(end+1)`.
- If used too often, e.g. in a loop, the price of **memory allocation** is high.

In [5]:
clear all;
x = rand (500_000,1);

In [6]:
function y = my_tan1 (x)
  y = [];
  for i = 1:numel (x)
    y(end+1) = sin (x(i)) / cos (x(i));
  endfor
endfunction

In [7]:
tic;
my_tan1 (x);
t1 = toc

t1 =  11.170


In [8]:
function y = my_tan2 (x)
  y = zeros (size (x));
  for i = 1:numel (x)
    y(i) = sin (x(i)) / cos (x(i));
  endfor
endfunction

In [9]:
tic;
my_tan2 (x);
t2 = toc
printf("%.2f %%", (-1 + t2 / t1) * 100)

t2 =  8.1175
-27.33 %

In [10]:
tic;
tan (x);
t3 = toc
printf("%.2f %%", (-1 + t3 / t1) * 100)

t3 =  0.030731
-99.72 %

## In-Place Operations (Octave only!)

In [11]:
clear all;
n = 20_000;
A = rand (n, n);

In [12]:
tic;
A = A + 1;
t1 = toc

t1 =  0.69806


In [13]:
tic;
A += 1;
t3 = toc
printf("%.2f %%", (-1 + t3 / t1) * 100)

t3 =  0.40725
-41.66 %

## Rules of thumb for optimizing Octave code

1. Implement your algorithm **correct** and **readable**.
2. Use as much **builtin Octave functions** as possible,
   avoid code **interpretation**.
3. Find bottlenecks using `profile()`.
   Determine a **reference time** to compare optimization attempts.
   ```octave
      profile on;
      ## CODE_TO_INSPECT
      profile off;
      
      profshow ()
      profexplore ()
      profexport ("/path/to/html")
   ```
   Making code more difficult to read **must** pay off!
4. **Vectorize** Matrix-Vector operations wherever possible.
5. Avoid `y(end+1)`, **preallocate memory** `y = zeros(...)`.
6. Avoid large **intermediate copies** of data.

If performance is still an issue: use the
[external code interface](https://octave.org/doc/v5.1.0/External-Code-Interface.html).

## References

- Rik Wehbring (2015).  Writing High Performance m-files.
  URL https://wiki.octave.org/File:High_Performance_Mfiles.pdf
- Vectorization and Faster Code Execution.
  In: John W. Eaton, David Bateman, Søren Hauberg, Rik Wehbring (2019).
  GNU Octave version 5.1.0 manual: a high-level interactive language for
  numerical computations.
  URL https://octave.org/doc/v5.1.0/Vectorization-and-Faster-Code-Execution.html